Hive is an Interface used in HADOOP Distributed File System(HDFS).Hive allows us to think of our files in HDFS as a database.
Normally we use a type of SQL command (not MySQL) under HIVE in HDFS.

R is used to import and analyze single dataset. To use multiple datasets we import them in a database like MySQL. But if its very large data(single or Multiple) we need use HDFS or SPARK.

We will get Hindersight of a HR analytics datasets by Hive in HDFS.
We will Perform a range of analytical queries using SQL to gain hindsight from a
given dataset(s). These will be performed using HIVE and HDFS. 

1. From PuTTY (external) --> connect to Remote.Labs:
# ssh s1234567@remote.labs.eait.uq.edu.au

2. From the Remote.Labs session --> connect to clientnode
# ssh s1234567@clientnode.zones.eait.uq.edu.au
ClientNode is where:
it's a Linux system connected to a Hadoop Namenode somewhere.

3.We can download the HR Analytics dataset onto the client node using the wget program.
wget https://stluc.manta.uqcloud.net/mdatascience/public/datasets/HumanResourceAnalytics/HR_comma_sep.csv

4.So far we have downloaded the HR Analytics Dataset to the local filesystem. To be able to use the Hadoop tools, we first have to push the file from our local filesystem into HDFS.
To push a local file into HDFS, we can use:

hadoop fs -put [local path] [hdfs path]

As long as we have completed the previous step and downloaded the HR Analytics, we could push it into our home directory in HDFS with the following command:

$ hadoop fs -put HR_comma_sep.csv



we can begin Hive by typing "hive" into the command line prompt. It may take a few seconds to initialise. When it's ready to receive commands, you should see a prompt like the following:

hive>
Hive allows us to think of our files in HDFS as a database, and query it in a similar way we would in MySQL. In order to create new tables, you will first need to connect to your database. Connect to your database using the following command, where the sXXXXXX is replaced with your student number.

hive> use sXXXXXXX;
If working, Hive should return an "OK" message along with how long the query took to make.

Now that we are working within the correct database, the next step will be to load the HR Dataset we previously pushed into HDFS into a temporary table, called tmp_hr. First we will need to create the temporary table using the following syntax:

create table tmp_hr (col_value STRING);
Once we've created the table, we can import our HR Analytics dataset from HDFS into our newly created table using the following command, replacing sXXXXXX with your student number:

LOAD DATA INPATH '/user/sXXXXXX/HR_comma_sep.csv' OVERWRITE INTO TABLE tmp_hr;
You may have noticed that the above statement doesn't include any information about the column types in our HR Analytics dataset. We first create a temporary table to consume the data into Hive. The temporary table contains only one column which is treated as a string. As you would have learned from last weeks prac, we are limited with what we can do with just a string data type. Let's create a more structured table that informs Hive about what each columns data type is in our HR Analytics dataset.

First, we need to create a new table that has the appropriate structure. We need to know what the column headers are, as well as the data types for those columns. Enter the following command into Hive to create a new table called hr with the appropriate data types.

CREATE TABLE HR (satisfaction_level FLOAT, last_evaluation FLOAT, number_project INT, average_monthly_hours INT, time_spend_company INT, Work_accident INT,left_job INT, promotion_last_5_years INT, sales STRING, salary STRING);

Now we have a new empty table with the required structure, but we still need to import the data from our temporary table into our new table. Enter in the following command to extract the appropriate parts from the tmp_hr table and insert them into our structured table.

INSERT OVERWRITE TABLE hr
SELECT
  regexp_extract(col_value, '^(?:([^,]*),?){1}', 1) satisfaction_level,
  regexp_extract(col_value, '^(?:([^,]*),?){2}', 1) last_evaluation,
  regexp_extract(col_value, '^(?:([^,]*),?){3}', 1) number_project,
  regexp_extract(col_value, '^(?:([^,]*),?){4}', 1) average_monthly_hours,
  regexp_extract(col_value, '^(?:([^,]*),?){5}', 1) time_spend_company,
   regexp_extract(col_value, '^(?:([^,]*),?){6}', 1) Work_accident,
  regexp_extract(col_value, '^(?:([^,]*),?){7}', 1) left_job,
  regexp_extract(col_value, '^(?:([^,]*),?){8}', 1) promotion_last_5_years,
  regexp_extract(col_value, '^(?:([^,]*),?){9}', 1) sales,
  regexp_extract(col_value, '^(?:([^,]*),?){10}', 1) salary
FROM tmp_hr;       

In our above statement, the regular expression is the string "(?:([^,]*),?){1}, as you may have guessed, this statement matches any strings between the comma character. The {1} refers to the nth match that is used to fill the specified column. For example: regexp_extract(col_value, '^(?:([^,]*),?){1}', 1) satisfaction_level extracts the first match of the regular expression, and places it into the satisfaction_level column.

Now that we have ingested our data into a new, structured table called hr, we can query the table exactly like we did in phpMyAdmin, using SQL. As an interface, you will see that Hive is very similar to a traditional database. However, instead of using a database backend, Hive can abstract our queries over many machines. Although not obvious in a dataset this size, if we had a dataset that was several terabytes in size, you would see significant performance gain over a tradition database as it is backed by a distributed file system and the queries can be distributed across several machines.

Answer the following questions and **include the SQL query used to determine the answer**.

Unlike phpMyAdmin, you will need to ensure that you end each statement with a ';'.

|<center>TASK</center>|
| ---- |
| How many entries were there in the HR dataset? |

15000 entries

SELECT COUNT(*) FROM HR;

Unlike MySQL, we can see that some of our queries were submitted as jobs, and may have taken a few seconds to process. Jobs are a common way of describing how processing is submitted to large distributed systems. Unlike our phpMyAdmin database that was only used by us, large distributed systems are often shared and processing may take hours or months. This means that often a job has to be submitted, picked up by some sort of job management process, directed to the appropriate node/s, distribute processing across multiple jobs, maintain a job queue and collate the results across multiple nodes (among many other things!). we can see why this complexity adds time - especially since many of these interactions are over a network. The benefits of a distributed system are only apparent when our dataset is big enough to warrant using one. In our case of our small HR Dataset, it is actually much faster to use R or MySQL!